In [215]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.utils import to_categorical
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.initializers import glorot_normal
from keras.utils import np_utils
from math import ceil

### Training data consists of 20000 numbers in binary representation(64 bits) and test contains 1000 samples

In [125]:
n_bits=64

In [194]:
X_train=np.load('xtrain.npy').reshape(-1,n_bits)
y_train=np.load('ytrain.npy').reshape(-1,1)
X_test=np.load('xtest.npy').reshape(-1,n_bits)
y_test=np.load('ytest.npy').reshape(-1,1)

### Testing out the simplest possible model i.e single layer

In [241]:
model = Sequential()
model.add(Dense(1,activation='relu',input_shape=(n_bits,)))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_62"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_160 (Dense)            (None, 1)                 65        
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


#### Model has only 65 parameters, 1 weight for each bit and a bias value

In [242]:
model.fit(X_train,y_train,epochs=25)

Epoch 1/25
9000/9000 [==============================] - 1s 127us/step - loss: 4.0054 - accuracy: 0.4779
Epoch 2/25
9000/9000 [==============================] - 1s 79us/step - loss: 3.2825 - accuracy: 0.5142
Epoch 3/25
9000/9000 [==============================] - 1s 76us/step - loss: 2.7511 - accuracy: 0.5426
Epoch 4/25
9000/9000 [==============================] - 1s 77us/step - loss: 2.4411 - accuracy: 0.5854
Epoch 5/25
9000/9000 [==============================] - 1s 88us/step - loss: 2.0888 - accuracy: 0.5959
Epoch 6/25
9000/9000 [==============================] - 1s 73us/step - loss: 1.9393 - accuracy: 0.6170
Epoch 7/25
9000/9000 [==============================] - 1s 74us/step - loss: 1.6005 - accuracy: 0.6451
Epoch 8/25
9000/9000 [==============================] - 1s 76us/step - loss: 1.4423 - accuracy: 0.6537
Epoch 9/25
9000/9000 [==============================] - 1s 72us/step - loss: 1.1280 - accuracy: 0.6900
Epoch 10/25
9000/9000 [==============================] - 1s 72us/step - 

### Model sometimes gets stuck on local optima and weights need to be reinitialized

In [243]:
model.evaluate(X_test,y_test)

1000/1000 [==============================] - 0s 452us/step


[0.050511517450213435, 1.0]

In [244]:
for layer in model.layers:
    print(layer.get_weights())

[array([[ 0.00677363],
       [-0.02263455],
       [ 0.02951043],
       [ 0.01654496],
       [ 0.03770161],
       [ 0.04202359],
       [-0.01287957],
       [ 0.02092279],
       [ 0.05119466],
       [ 0.05705069],
       [-0.01606347],
       [ 0.02950711],
       [ 0.04261974],
       [ 0.03060959],
       [ 0.04790935],
       [ 0.0728935 ],
       [ 0.01675704],
       [ 0.0339079 ],
       [ 0.05075683],
       [ 0.00910498],
       [ 0.03396915],
       [ 0.04988522],
       [ 0.03190381],
       [-0.01481624],
       [ 0.04096314],
       [ 0.05275473],
       [ 0.04637457],
       [ 0.05065171],
       [ 0.06772818],
       [ 0.02853998],
       [ 0.03536513],
       [ 0.06779679],
       [ 0.03057687],
       [ 0.03808122],
       [ 0.05517583],
       [ 0.03946064],
       [ 0.01508139],
       [ 0.03320685],
       [-0.01026512],
       [-0.00829757],
       [ 0.0567228 ],
       [ 0.03073649],
       [-0.00155868],
       [ 0.04311327],
       [ 0.04249937],
       [ 

### As expected we can see the model has learned to use the last bit in predicting odd/even
**The last neuron has a weight of -1.09 i.e when the last bit is 1 the negative weight followed by the sigmoid makes the model output 0 corresponding to a odd number and similarly 1 for even number**

### Adding another intermediate layer of 64 neurons

In [257]:
model = Sequential()

model.add(Dense(64,activation='tanh',input_shape=(n_bits,)))
model.add(Dense(1,activation='relu'))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_169 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_170 (Dense)            (None, 1)                 65        
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


In [258]:
model.fit(X_train,y_train,epochs=25)

Epoch 1/25
9000/9000 [==============================] - 1s 132us/step - loss: 0.8291 - accuracy: 0.8347
Epoch 2/25
9000/9000 [==============================] - 1s 80us/step - loss: 0.0241 - accuracy: 0.9994
Epoch 3/25
9000/9000 [==============================] - 1s 81us/step - loss: 0.0037 - accuracy: 0.9980
Epoch 4/25
9000/9000 [==============================] - 1s 79us/step - loss: 0.0013 - accuracy: 0.9969
Epoch 5/25
9000/9000 [==============================] - 1s 80us/step - loss: 6.4215e-04 - accuracy: 0.9941
Epoch 6/25
9000/9000 [==============================] - 1s 81us/step - loss: 3.4332e-04 - accuracy: 0.9933
Epoch 7/25
9000/9000 [==============================] - 1s 82us/step - loss: 1.8786e-04 - accuracy: 0.9898
Epoch 8/25
9000/9000 [==============================] - 1s 81us/step - loss: 1.1200e-04 - accuracy: 0.9898
Epoch 9/25
9000/9000 [==============================] - 1s 80us/step - loss: 9.2408e-05 - accuracy: 0.9857
Epoch 10/25
9000/9000 [=============================

In [261]:
model.evaluate(X_test,y_test)

1000/1000 [==============================] - 0s 96us/step


[0.0, 0.9580000042915344]

In [262]:
for layer in model.layers:
    print(layer.get_weights())

[array([[-0.18183008, -0.06281184,  0.21292016, ...,  0.00548634,
         0.13471358, -0.02666843],
       [ 0.08163736, -0.06878897, -0.1251381 , ..., -0.1165362 ,
        -0.0399698 ,  0.11253046],
       [-0.00192882, -0.111926  , -0.17232656, ...,  0.09507027,
        -0.00426782, -0.01518515],
       ...,
       [-0.03900824, -0.11499169, -0.04758181, ...,  0.0968859 ,
        -0.05187183, -0.05518178],
       [ 0.07416444,  0.08138244, -0.10705304, ..., -0.12463407,
        -0.0956605 ,  0.13453257],
       [ 0.28112248, -0.03939004,  0.23373301, ..., -0.03781975,
        -0.2638293 , -0.21135405]], dtype=float32), array([-0.02297718,  0.02226296, -0.02515272,  0.07343481, -0.02527467,
       -0.02736185,  0.02960105,  0.03894471, -0.03244314,  0.01347534,
       -0.00610123,  0.06319445, -0.04692763, -0.02317805,  0.024689  ,
       -0.05424469, -0.05036336, -0.05107069,  0.01814247, -0.02144654,
        0.02535893,  0.01928585, -0.01577857, -0.05941382,  0.01870255,
        0.

In [199]:
model = Sequential()
model.add(Dense(1024, activation='tanh',input_shape=(n_bits,)))
model.add(Dense(512,activation='tanh'))
model.add(Dense(512,activation='tanh'))
model.add(Dense(512,activation='tanh'))
model.add(Dense(1,activation='relu'))
adam=keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_147 (Dense)            (None, 1024)              66560     
_________________________________________________________________
dense_148 (Dense)            (None, 512)               524800    
_________________________________________________________________
dense_149 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_150 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_151 (Dense)            (None, 1)                 513       
Total params: 1,117,185
Trainable params: 1,117,185
Non-trainable params: 0
_________________________________________________________________


In [200]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
9000/9000 [==============================] - 6s 622us/step - loss: 0.3688 - accuracy: 0.8672
Epoch 2/10
9000/9000 [==============================] - 5s 503us/step - loss: 0.0132 - accuracy: 0.9999
Epoch 3/10
9000/9000 [==============================] - 4s 499us/step - loss: 0.0020 - accuracy: 0.9979
Epoch 4/10
9000/9000 [==============================] - 5s 506us/step - loss: 7.6838e-04 - accuracy: 0.9927
Epoch 5/10
9000/9000 [==============================] - 5s 501us/step - loss: 3.7342e-04 - accuracy: 0.9829
Epoch 6/10
9000/9000 [==============================] - 4s 497us/step - loss: 1.9054e-04 - accuracy: 0.9750
Epoch 7/10
9000/9000 [==============================] - 5s 508us/step - loss: 1.0518e-04 - accuracy: 0.9667
Epoch 8/10
9000/9000 [==============================] - 4s 499us/step - loss: 6.6774e-05 - accuracy: 0.9538
Epoch 9/10
9000/9000 [==============================] - 4s 497us/step - loss: 4.2090e-05 - accuracy: 0.9497
Epoch 10/10
9000/9000 [=================

In [260]:
model.evaluate(X_test,y_test)

1000/1000 [==============================] - 0s 91us/step


[0.0, 0.9580000042915344]

In [202]:
model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_147 (Dense)            (None, 1024)              66560     
_________________________________________________________________
dense_148 (Dense)            (None, 512)               524800    
_________________________________________________________________
dense_149 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_150 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_151 (Dense)            (None, 1)                 513       
Total params: 1,117,185
Trainable params: 1,117,185
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.set_weights([np.array([0,0,0,0,0,0,0,1]).reshape(-1,1),[0]])